In [70]:
import pandas as pd
import numpy as np
import os
import glob

In [71]:
path = os.getcwd()

# read all the files with extension .csv
filenames = glob.glob(os.path.join(path, "*.txt"))

In [119]:
SubjectContainer = []
for file in filenames:
    
    Subject = file.split('_')[0]
    Subject = Subject.split('\\')[-1]
    d=pd.read_csv(file, skiprows= 18, header=None, delimiter='\t',  escapechar='\\')

    #identify break of dataframe by Total verbiage
    getBreak = d[d[0]=='Total']
    getBreak = getBreak[[0]].assign(df_Break=range(len(getBreak)))
    d = pd.merge(d, getBreak, how='left', left_index=True, right_index=True)
    d['df_Break'] = d['df_Break'].bfill()
    d['df_Break'] = d['df_Break'].astype(int)

    #for each dataframe make data long
    grouped = d.groupby('df_Break')
    
    StageContainer = []
    for name, group in grouped:
        
        #remove not needed header details
        removeHeaderData = group["0_x"].str.contains('Time Division')==False
        group = group[removeHeaderData]


        #drop break column
        Session = group.drop('df_Break', axis=1)

        #Session.to_csv(Subject +"test.csv")
        Session.reset_index(drop=True, inplace=True)

        #get stage data
        getStage = Session[2][0]
        Session = Session.drop(Session.index[[0,1]])

        #drop junk column
        Session = Session.drop('0_y', axis=1)

        ColumnHeader = Session['0_x']
        Session = Session.drop('0_x', axis=1)

        #convert to numeric
        Session = Session.astype(np.float64)

        Session = Session.sum(numeric_only=True,axis=1)

        Session = pd.merge(ColumnHeader.to_frame(), Session.to_frame(), how='inner', left_index=True, right_index=True)
        
        Session = Session.transpose()
        Session.columns = Session.iloc[0]
        Session = Session.iloc[1:].reset_index(drop=True)



        Session.to_csv("test.csv")

        print(Session)
        Session['Stage'] = getStage

        Session['Subject'] = Subject
        StageContainer.append(Session)

    StagedData = pd.concat(StageContainer)

    SubjectContainer.append(StagedData)

FinalSet = pd.concat(SubjectContainer)


0_x  0.000000  0.976563  1.953125  2.929688  3.906250  4.882813  5.859375   
0    0.000657  0.003204  0.004274  0.007104  0.007557  0.004053  0.002261  \

0_x  6.835938  7.812500  8.789063  ... 124.023438 125.000000  EpochNo   
0    0.001471  0.000944  0.000621  ...        0.0        0.0  11704.0  \

0_x Discarded     Delta     Theta     Alpha    Gammas   GammaF     Total  
0         0.0  0.005534  0.001632  0.000264  0.000157  0.00006  0.000314  

[1 rows x 137 columns]
0_x  0.000000  0.976563  1.953125  2.929688 3.906250  4.882813  5.859375   
0    0.000142  0.000685  0.001478  0.002349  0.00135  0.000748  0.000796  \

0_x  6.835938  7.812500  8.789063  ... 124.023438 125.000000 EpochNo   
0    0.001202  0.001323  0.000911  ...        0.0        0.0   438.0  \

0_x Discarded     Delta     Theta     Alpha    Gammas    GammaF     Total  
0         0.0  0.001465  0.000924  0.000222  0.000081  0.000018  0.000121  

[1 rows x 137 columns]
0_x  0.000000  0.976563  1.953125  2.929688  3.906

In [120]:
FinalSet = FinalSet.set_index(["Subject", "Stage"])

In [122]:
FinalSet.to_csv("Concatenated Data.csv", index=True)